<a href="https://colab.research.google.com/github/kenhuangsy/learning-tensorflow/blob/main/04_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Building and fitting a model using the same data but with MobileNetV2 architecture feature extraction



## Downloading the data

In [1]:
import zipfile

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()

--2022-11-14 02:14:39--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.119.128, 108.177.126.128, 108.177.127.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.119.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M  38.2MB/s    in 4.8s    

2022-11-14 02:14:44 (33.6 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [2]:
import os

os.listdir()

['.config',
 '__MACOSX',
 '10_food_classes_10_percent.zip',
 '10_food_classes_10_percent',
 'sample_data']

In [3]:
os.walk

<function os.walk(top, topdown=True, onerror=None, followlinks=False)>

In [4]:
import os

for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in {dirpath}.")

There are 2 directories and 0 images in 10_food_classes_10_percent.
There are 10 directories and 0 images in 10_food_classes_10_percent/train.
There are 0 directories and 75 images in 10_food_classes_10_percent/train/hamburger.
There are 0 directories and 75 images in 10_food_classes_10_percent/train/steak.
There are 0 directories and 75 images in 10_food_classes_10_percent/train/chicken_wings.
There are 0 directories and 75 images in 10_food_classes_10_percent/train/chicken_curry.
There are 0 directories and 75 images in 10_food_classes_10_percent/train/ice_cream.
There are 0 directories and 75 images in 10_food_classes_10_percent/train/sushi.
There are 0 directories and 75 images in 10_food_classes_10_percent/train/fried_rice.
There are 0 directories and 75 images in 10_food_classes_10_percent/train/ramen.
There are 0 directories and 75 images in 10_food_classes_10_percent/train/pizza.
There are 0 directories and 75 images in 10_food_classes_10_percent/train/grilled_salmon.
There are

## Creating data loaders (preparing the data)

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32

train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

train_datagen = ImageDataGenerator(rescale = 1/255.)
test_datagen = ImageDataGenerator(rescale = 1/255.)

print("Training images:")
train_data_10_percent = train_datagen.flow_from_directory(train_dir,
                                                          target_size = IMAGE_SHAPE,
                                                          batch_size = BATCH_SIZE,
                                                          class_mode = "categorical")

print("Testing images:")
test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size = IMAGE_SHAPE,
                                             batch_size = BATCH_SIZE,
                                             class_mode = "categorical")

Training images:
Found 750 images belonging to 10 classes.
Testing images:
Found 2500 images belonging to 10 classes.


### Setting up callbacks

In [6]:
import tensorflow as tf

import datetime as dt

def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + dt.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

### Creating model

In [7]:
mobilenet_url = "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/5"


In [8]:
import tensorflow_hub as hub
from tensorflow.keras import layers

In [9]:
def create_model(model_url: str, num_classes = 10) -> tf.keras.Sequential:
  """
  Takes a TensorFlow Hub URL and creataes a keras sequential model with it

  Args:
    model_url (str): A tensorflow hub feature extraction URL
    num_classes (int): Number of output neurons in the output layer = 10 as default

  Returns:
    An uncompiled Keras Sequential model with model_url as feature extractor layer
    and Dense output layer with num_classes output neurons
  """
  feature_extraction_layer = hub.KerasLayer(model_url,
                                            trainable = False,
                                            name = "feature_extraction_layer",
                                            input_shape = IMAGE_SHAPE + (3,))
  
  model = tf.keras.Sequential([
      feature_extraction_layer,
      layers.Dense(num_classes, activation = "softmax", name = "output_layer")
  ])
  return model

In [10]:
# Create mobilenet model

mobilenet_model = create_model(mobilenet_url)

In [11]:
mobilenet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_extraction_layer (K  (None, 1280)             2257984   
 erasLayer)                                                      
                                                                 
 output_layer (Dense)        (None, 10)                12810     
                                                                 
Total params: 2,270,794
Trainable params: 12,810
Non-trainable params: 2,257,984
_________________________________________________________________


In [12]:
# Compile our model
mobilenet_model.compile(loss = "categorical_crossentropy",
                        optimizer = tf.keras.optimizers.Adam(),
                        metrics= ["accuracy"])

In [17]:
# Fit our model
mobilenet_history = mobilenet_model.fit(train_data_10_percent,
                                        epochs = 10,
                                        steps_per_epoch = len(train_data_10_percent),
                                        validation_data = test_data,
                                        validation_steps = len(test_data),
                                        callbacks = [create_tensorboard_callback(dir_name = "tensorflow_hub",
                                                                                 experiment_name = "mobilenet_v2")])

Saving TensorBoard log files to: tensorflow_hub/mobilenet_v2/20221114-022305
Epoch 1/10
24/24 [==============================] - 15s 634ms/step - loss: 0.2062 - accuracy: 0.9813 - val_loss: 0.5628 - val_accuracy: 0.8196
Epoch 2/10
24/24 [==============================] - 15s 665ms/step - loss: 0.1849 - accuracy: 0.9853 - val_loss: 0.5633 - val_accuracy: 0.8160
Epoch 3/10
24/24 [==============================] - 15s 630ms/step - loss: 0.1641 - accuracy: 0.9920 - val_loss: 0.5552 - val_accuracy: 0.8204
Epoch 4/10
24/24 [==============================] - 15s 631ms/step - loss: 0.1477 - accuracy: 0.9933 - val_loss: 0.5540 - val_accuracy: 0.8188
Epoch 5/10
24/24 [==============================] - 15s 662ms/step - loss: 0.1345 - accuracy: 0.9933 - val_loss: 0.5513 - val_accuracy: 0.8188
Epoch 6/10
24/24 [==============================] - 15s 626ms/step - loss: 0.1205 - accuracy: 0.9947 - val_loss: 0.5517 - val_accuracy: 0.8196
Epoch 7/10
24/24 [==============================] - 15s 628ms/ste

In [18]:
!tensorboard dev upload --logdir ./tensorflow_hub/ \
  --name "MobileNetV2" \
  --description "MobileNetV2 performance" \
  --one_shot


New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/kIgyUnBoQ3qmXgAaonn2kw/

[2022-11-14T02:26:03] Started scanning logdir.
[2022-11-14T02:26:05] Total uploaded: 60 scalars, 0 tensors, 1 binary objects (1.6 MB)
[2022-11-14T02:26:05] Done scanning logdir.


Done. View your TensorBoard at https://tensorboard.dev/experiment/kIgyUnBoQ3qmXgAaonn2kw/
